# Project
playground for testing workflows

# Data set setup

In [2]:
import scipy.io

# prereqs import
import pandas as pd
import numpy as np
import math
import re
#from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt

In [3]:
# load matlab data
mat = scipy.io.loadmat('matlab/emnist-digits')
mat.keys()
data = mat['dataset']


In [4]:
# see page 4 of the paper on emnist
# uncertain what 3rd entry is in dataset
X_train = data['train'][0,0]['images'][0,0]
y_train = data['train'][0,0]['labels'][0,0]
X_test = data['test'][0,0]['images'][0,0]
y_test = data['test'][0,0]['labels'][0,0]

In [13]:
# try proccessing data (pixel values from EMINST are not normalized, 0-255)
def rescale_DivideBy255(inputData):
    outputArray = []
    for i in range(len(inputData)):
        outputArray.append(inputData[i]/255)

    return outputArray

X_train_normalized = rescale_DivideBy255(X_train)
X_test_normalized = rescale_DivideBy255(X_test)

In [14]:
# For digits expect 280,000 total samples, 240,000 for training, 40,000 for testing. 
len(X_train_normalized)

240000

In [15]:
#73~ minutes to train on non normalized data
# 9 minute train time normalized data
from sklearn import neural_network

testNN = neural_network.MLPClassifier()
testNN.fit(X_train_normalized,y_train)

c:\Users\hunter\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1102: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier()

In [16]:
# 0.97 accuracy classifying non normalized data
# 0.9862 accuracy classifying normalized data
testNN.score(X_test_normalized,y_test)

0.9862